In [127]:
import numpy as np
from typing import Callable

In [128]:
def parse_input(input_file: str) -> tuple[list]:
    with open(input_file) as dots_and_folds:
        dots_and_folds = dots_and_folds.readlines()
    dots = [(int(y), int(x)) for dot in dots_and_folds if (coords := dot.strip().split(',')) and coords[0].isnumeric() for x, y in [coords]]
    folds = [tuple(fold.strip().removeprefix('fold along ').split('=')) for fold in dots_and_folds if fold[0].isalpha()]
    return dots, folds

In [129]:
def create_inital_empty_grid(coords: list[tuple[int, int]]) -> np.ndarray:
    y_coords, x_coords = zip(*coords)
    max_x_coord = max(x_coords)
    max_y_coord = max(y_coords)
    return np.zeros((max_y_coord + 1, max_x_coord + 1))

In [130]:
def create_empty_grid_for_fold(current_grid: np.ndarray, fold_axis: str, fold_coord: int) -> np.ndarray:
    if fold_axis == 'y':
        return np.zeros((fold_coord, current_grid.shape[1]))
    if fold_axis == 'x':
        return np.zeros((current_grid.shape[0], fold_coord))

In [131]:
def add_dots_to_grid(coords: list[tuple[int, int]], empty_array: np.ndarray):
    for coord in coords:
        empty_array[coord] = 1

In [132]:
def map_single_coord(fold_axis: str, fold_coord: int) -> Callable:
    def map_on_y_fold(coord: tuple[int, int]) -> tuple[int, int]:
        return (2*fold_coord - coord[0], coord[1])
    def map_on_x_fold(coord: tuple[int, int]) -> tuple[int, int]:
        return (coord[0], 2*fold_coord - coord[1])
    if fold_axis == 'y':
        return map_on_y_fold
    if fold_axis == 'x':
        return map_on_x_fold

In [133]:
def find_and_map_coords_on_fold(coords: list[tuple[int, int]], fold_axis: str, fold_coord: int) -> list[tuple[int, int]]:
    i = 0 if fold_axis == 'y' else 1
    coords_to_map = [coord for coord in coords if coord[i] > fold_coord]
    map_on_y_fold = map_single_coord(fold_axis, fold_coord)
    mapped_coords = set(map(map_on_y_fold, coords_to_map))
    updated_coords = (set(coords) - set(coords_to_map)) | mapped_coords
    updated_coords = [coord for coord in updated_coords if coord[i] != fold_coord]
    return updated_coords

In [134]:
def fold_paper_once(input_file: str) -> int:
    dots, folds = parse_input(input_file)
    fold_axis = folds[0][0]
    fold_coord = int(folds[0][1])
    dots_after_first_fold = find_and_map_coords_on_fold(dots, fold_axis, fold_coord)
    return len(dots_after_first_fold)

In [135]:
fold_paper_once('practise_input.txt')

17

In [136]:
fold_paper_once('real_input.txt')

942

Part 2

In [137]:
def make_grid_readable(grid: np.ndarray) -> list[str]:
    readable_grid = []
    for line in grid:
        readable_line = []
        for item in line:
            if item == 0:
                readable_line.append('.')
            if item == 1:
                readable_line.append('#')
        readable_grid.append(readable_line)
    return readable_grid

In [138]:
def find_final_code(input_file: str) -> int:
    dots, folds = parse_input(input_file)
    current_grid = create_inital_empty_grid(dots)
    for fold in folds:
        fold_axis = fold[0]
        fold_coord = int(fold[1])
        dots = find_and_map_coords_on_fold(dots, fold_axis, fold_coord)
        current_grid = create_empty_grid_for_fold(current_grid, fold_axis, fold_coord)
        add_dots_to_grid(dots, current_grid)
    return make_grid_readable(current_grid)

In [139]:
find_final_code('practise_input.txt')

[['#', '#', '#', '#', '#'],
 ['#', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '#'],
 ['#', '#', '#', '#', '#'],
 ['.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.']]

In [140]:
readable_grid = find_final_code('real_input.txt')

In [141]:
for line in readable_grid:
    print(line)

['.', '.', '#', '#', '.', '#', '#', '#', '#', '.', '.', '#', '#', '.', '.', '#', '.', '.', '#', '.', '.', '#', '#', '.', '.', '#', '#', '#', '.', '.', '#', '#', '#', '.', '.', '#', '#', '#', '.', '.']
['.', '.', '.', '#', '.', '.', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.']
['.', '.', '.', '#', '.', '.', '.', '#', '.', '.', '#', '.', '.', '.', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '#', '#', '.', '.']
['.', '.', '.', '#', '.', '.', '#', '.', '.', '.', '#', '.', '#', '#', '.', '#', '.', '.', '#', '.', '#', '#', '#', '#', '.', '#', '#', '#', '.', '.', '#', '#', '#', '.', '.', '#', '.', '.', '#', '.']
['#', '.', '.', '#', '.', '#', '.', '.', '.', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.', '#', '.', '.', '.', '.', '#', '.', '#', '.', '.', '#', '.', '.', '#', 